In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, utils
from torch.autograd import Variable

torch.manual_seed(42)

# Step 1 Build data pipeline and import image dataset (split into training set and test set 5:5)

In [2]:
# data_dir is the address where you store your data data, I put it here under relative path
data_dir = "D:/zhuomian/2022_project/data"
split = 0.5
batch_size = 32
learning_rate = 0.001
num_epochs = 30

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [3]:
# Build data pre-processing
data_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Resize((56,56)),
                                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# Import data into dataset
dataset = datasets.ImageFolder(root = data_dir, transform = data_transform)

# Now divide the data set into a training set and a test set
len_imgs = len(dataset.imgs)
train_set, test_set = torch.utils.data.random_split(dataset, [int(len_imgs*split), len_imgs-int(len_imgs*split)]) 

# Final construction of the data pipeline
train_loader = torch.utils.data.DataLoader(train_set,batch_size = batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size = batch_size,shuffle = False)

# Step 2 Build the model Customized loss function and optimizer

In [4]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        # conv2d Parameters nn.Conv2d(in_channels=3, out_channels=32,kernel_size=5,stride=1,padding="same")
        # MaxPool2d Parameters nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv01 = nn.Sequential(         
            nn.Conv2d(64, 96, 1, 1, padding = "same"),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.Conv2d(96, 96, 3, 1, padding = "same"),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.Conv2d(96, 64, 1, 1, padding = "same"),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.conv02 = nn.Sequential(         
            nn.Conv2d(32, 32*6, 1, 1, padding = "same"),
            nn.BatchNorm2d(32*6),
            nn.ReLU(),
            nn.Conv2d(32*6, 32*6, 3, 1, padding = "same"),
            nn.BatchNorm2d(32*6),
            nn.ReLU(),
            nn.Conv2d(32*6, 32, 1, 1, padding = "same"),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )
        self.conv1 = nn.Sequential(         
            nn.Conv2d(3, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2) 
        )
        
        self.conv2 = nn.Sequential(         
            nn.Conv2d(32, 64, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        self.conv3 = nn.Sequential(         
            nn.Conv2d(64, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        # Fully connected layer with 21 output categories
        self.fc1 = nn.Sequential(nn.Linear(32 * 7 * 7, 128),nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(128, 21))
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        identity = x
        x = self.conv01(x)
        x = identity + x
        x = self.conv3(x)
        identity = x
        x = self.conv02(x)
        x = identity + x
        # Flattening treatment x.view
        x = x.view(x.size(0), -1)       
        x = self.fc1(x)
        output = self.fc2(x)
        return output    # return x for visualization

In [5]:
model = CNN_model()
model.to(device)
# Choose whether to print the model or not, here it will not be printed, just comment
print(model)

# Select loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate, betas=(0.9, 0.999),
                       eps=1e-07, amsgrad=False)

CNN_model(
  (conv01): Sequential(
    (0): Conv2d(64, 96, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (conv02): Sequential(
    (0): Conv2d(32, 192, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(192, 32, kernel_size=(1, 1

# Step 3 Pass in the model to start training and testing

In [6]:
# Train the model and record the accuracy and loss values for each epoch during training
def train(num_epochs, model, loaders):
    model.train()   
    
    # The first stores the correct rate and the second stores the loss value
    Accy_list = []
    Loss_list = []
    
    total_step = len(loaders)    
    for epoch in range(num_epochs):
        correct = 0
        L = 0
        for i, (images, labels) in enumerate(loaders):
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            loss = loss_func(outputs, labels)
            
            # Clear the previous gradient 
            optimizer.zero_grad()           
            
            # Backpropagate and update the parameters
            loss.backward()               
            optimizer.step()                
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum()
            L += loss.item()
            if (i+1) % 9 == 0:
                print ('Epoch: [{}/{}]\t|\tStep: [{}/{}]\t|\tLoss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                
        accuracy = 100 * correct / len(train_set)
        print(accuracy.item())
        print("epoch: {:02d} | Accuracy: {:.2f} Loss: {:.4f}\n"
              .format(epoch + 1, accuracy, L))
        
        # Add the correct and lost values for this epoch to the corresponding list
        Accy_list.append(accuracy.item())
        Loss_list.append(L)
              
    return Accy_list, Loss_list


Accy_list, Loss_list = train(num_epochs, model, train_loader)

Epoch: [1/30]	|	Step: [9/27]	|	Loss: 3.0552
Epoch: [1/30]	|	Step: [18/27]	|	Loss: 2.9765
Epoch: [1/30]	|	Step: [27/27]	|	Loss: 3.1126
7.253270149230957
epoch: 01 | Accuracy: 7.25 Loss: 81.8540

Epoch: [2/30]	|	Step: [9/27]	|	Loss: 2.9095
Epoch: [2/30]	|	Step: [18/27]	|	Loss: 2.5404
Epoch: [2/30]	|	Step: [27/27]	|	Loss: 2.5528
20.808561325073242
epoch: 02 | Accuracy: 20.81 Loss: 71.0154

Epoch: [3/30]	|	Step: [9/27]	|	Loss: 2.2127
Epoch: [3/30]	|	Step: [18/27]	|	Loss: 1.8080
Epoch: [3/30]	|	Step: [27/27]	|	Loss: 1.9743
36.741973876953125
epoch: 03 | Accuracy: 36.74 Loss: 55.5331

Epoch: [4/30]	|	Step: [9/27]	|	Loss: 1.7678
Epoch: [4/30]	|	Step: [18/27]	|	Loss: 1.2737
Epoch: [4/30]	|	Step: [27/27]	|	Loss: 1.1944
57.193817138671875
epoch: 04 | Accuracy: 57.19 Loss: 38.2794

Epoch: [5/30]	|	Step: [9/27]	|	Loss: 1.1735
Epoch: [5/30]	|	Step: [18/27]	|	Loss: 0.9109
Epoch: [5/30]	|	Step: [27/27]	|	Loss: 1.0583
72.29488372802734
epoch: 05 | Accuracy: 72.29 Loss: 24.9142

Epoch: [6/30]	|	Step: [

In [7]:
print(Accy_list)

[7.253270149230957, 20.808561325073242, 36.741973876953125, 57.193817138671875, 72.29488372802734, 81.5695571899414, 90.36860656738281, 93.935791015625, 97.50297546386719, 97.85968780517578, 99.04875183105469, 99.7621841430664, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]


In [8]:
print(Loss_list)

[81.8540415763855, 71.01537132263184, 55.53305232524872, 38.27935338020325, 24.914187848567963, 15.970443278551102, 8.98940734565258, 6.109656281769276, 3.307019054889679, 2.889338616281748, 1.7376581206917763, 0.8500401666387916, 0.5366792250424623, 0.2894756766036153, 0.1806304007768631, 0.12822506169322878, 0.08790551219135523, 0.072197022032924, 0.07166448282077909, 0.06017083034384996, 0.05264249537140131, 0.04706970788538456, 0.04710524232359603, 0.04815552965737879, 0.04049541079439223, 0.03714443166973069, 0.03577589016640559, 0.03242642170516774, 0.029510805616155267, 0.028037981828674674]


In [9]:
# Test models
def test(model, loaders):
    model.eval()
    with torch.no_grad():
        correct = 0
        L = 0
        for images, labels in loaders:
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            # Get the loss value and add it up
            loss = loss_func(outputs, labels)
            L += loss.item()
            
             # Get the correct number of predicted samples
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).float().sum()
            
    accuracy = (100 * correct / len(test_set))
    print("Test data | Accuracy: {:.2f} %, Loss: {:.4f} ".format(accuracy, L))
    
    return accuracy, L

Test_acc, Test_loss = test(model, test_loader)

Test data | Accuracy: 90.61 %, Loss: 10.1444 
